## 크롤링 1

- 유튜브를 크롤링하여 데이터를 획득
- 크롤링을 해야 이유는 유튜브 자체가 ajax를 이용한 방식이므로 사람의 손길이 필요한
  사이트
- 일반적인 html 자체에는 자체 정의 태그를 들을 혼용한다 => Angular.js를 부분 사용을 했다라고 판단됨.
- 일반적인 css selector의 규칙중에 id규칙(문서내에 고유하다)을 무시한다
  => 사용 주의
- 데이터 로드는 ajax를 통해서 사용자가 스크롤링을 해서 리스트의 끝에 도착할때 더보기(이런 기능)이 처리되므로, 전체 데이터르 수집하기 위해서는 js를 가동시켜야 한다
- 사용라이브러리는 selenium을 사용
- https://www.seleniumhq.org/
- 웹드라이버(여기서는 크롬을 사용)
  > https://sites.google.com/a/chromium.org/chromedriver/downloads  
  > 최신버전이 적용 않되는 혹은 버그가 있는 기능이 잇으므오, 혹시 않되면, 계속 버전을 내려가몃서 사용  
- API-DOCs
  > https://seleniumhq.github.io/selenium/docs/api/py/index.html 
  > https://selenium-python.readthedocs.io/
- install
  > conda install selenium  
  > or  
  > pip install selenium

In [167]:
from selenium import webdriver as wd

In [168]:
import urllib

In [169]:
# 업로드 순 정렬, 오늘 (현재 기준 24시간 이내 게시물)
# target_url = 'https://www.youtube.com/results?search_query={}&sp=CAISBAgCEAE%253D'
# 이주소는 어디서 가져 온거;? 
target_url = 'https://www.youtube.com/results?search_query={}&sp=CAISBAgCEAE%253D'

In [170]:
keyword ='갤럭시노트10'

In [171]:
target_url = target_url.format(urllib.parse.quote(keyword))
target_url

'https://www.youtube.com/results?search_query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%EB%85%B8%ED%8A%B810&sp=CAISBAgCEAE%253D'

In [172]:
# 웹드라이버 가동 => 여기서 다운(https://sites.google.com/a/chromium.org/chromedriver/downloads)
# driver = wd.Chrome('웹 드라이버 저장경로')
# 같은 폴더에 저장되어있으경우 => driver = wd.Chrome() 이렇게도 동작한다. 
driver = wd.Chrome('../tool/chromedriver_75.0.3770.140.exe')

In [173]:
#사이트 접속 
driver.get(target_url)

In [ ]:
# 스크롤을  쳐서 데이터를 다 로드 시킨다. 
# 데이터량이 엄청 많지 않을 때 가능하다
# ajax를 사용할 경우 서버가 반응해서 응답이 오고 
# 화면구성 할때까지 시간이 걸린다. 스크롤마다 대기시간이 필요하다 
for n in range(100):
    driver.execute_script('window.scrollBy(0,1000)')
    # 다음 작업을 코드적으로 강제 대기시킨것이다. 
    # 데이터 추출시에는 selenium에서 지원하는 waits기법을 사용하면 좋다
    time.sleep(2)

In [ ]:
# 영상 데이터 한줄 :  ytd-video-renderer
# 영상 메타 정보 : video-title

In [ ]:
videos = driver.find_elements_by_tag_name('ytd-video-renderer')
len(videos),type(videos)

In [ ]:
#driver.find_element_by_css_selector
for video in videos[:2]:
    # 영상의 타이틀
    print( video.find_element_by_id('video-title').get_attribute('title') )
    # 썸네임
    print( video.find_element_by_tag_name('img').get_attribute('src') )
    # 링크
    print( video.find_element_by_id('video-title').get_attribute('href') )
    # 작성자
    print( video.find_element_by_css_selector('#metadata-line>a').text)
    # 조회수 : metadata-line, # 작성시간
    for span in video.find_elements_by_css_selector('#metadata-line>span'):
        print( span.text)
    '''
    [
      { 'title':'xx', 'thum':'xx', 'link':'xx', 'author':'xx', 'views':x, 'time':''}
      ,{},{},...
    ]
    '''

In [ ]:
results = []
for video in videos:#[:2]:    
    dic = {
        'title':video.find_element_by_id('video-title').get_attribute('title')
        ,'thum':video.find_element_by_tag_name('img').get_attribute('src')
        ,'link':video.find_element_by_id('video-title').get_attribute('href')
        ,'author':video.find_element_by_css_selector('#byline>a').text
        ,'views':video.find_elements_by_css_selector('#metadata-line>span')[0].text
        ,'time':video.find_elements_by_css_selector('#metadata-line>span')[1].text
    }
    results.append( dic )
results

In [ ]:
for link in results[-2:-1]:
    print( link['link'] )
    
    # 해당 콘텐츠에 진입
    driver.get( link['link'] )
    # 잠시대기 -> 반드시 신규페이지에서는 대기
    print('5초 대기시도')
    driver.implicitly_wait(5)
    
    # 화면 스크롤 
    for n in range(3):
        print('화면스크롤 시도')
        driver.execute_script('window.scrollBy(0,1000)')
    print('5초 대기시도')
    time.sleep(5)
    # 후기 개수를 확인 -> 적정선까지 로드
    for cnt in driver.find_elements_by_id('count'):
        print( cnt.text )
        
    # 후기 채집 (작성자, 내용, 좋아요, 시간)
    print( '-'*20 )
    for comment in driver.find_elements_by_tag_name('ytd-comment-thread-renderer'):        
        print( comment.text )
        print( '-'*20 )

In [ ]:
# 1. 한글명은 동일하게 세팅되니까, replace로 걷어내는 방법
# 2. 정규식을 통해서 숫자만 남기고 모두 제거 
tmp = ['조회수 39,529회', '댓글 362개']
tmp[0].replace('조회수 ','').replace('회','')

In [ ]:
tmp[1].replace('댓글','').replace('개','')

In [ ]:
driver.close()
driver.quit()